In [1]:
import numpy as np 
import pandas as pd 
# from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
# from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
import json
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

Using TensorFlow backend.


In [2]:
from sklearn.ensemble import RandomForestClassifier

In [3]:
dataset = pd.read_csv('dataset/drop_dupa.tsv')
print(len(dataset))
# dataset = data[['new_phrase','Sentiment']]

95641


In [4]:
corpus = []
for i in range(len(dataset)):
    review = dataset.new_phrase[i].split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)
    if i%3000 == 0:
        print(i)

0
3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000
42000
45000
48000
51000
54000
57000
60000
63000
66000
69000
72000
75000
78000
81000
84000
87000
90000
93000


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray()
print( np.array(X))
check_shape = np.array(X)
print('Shape of the Count Vectorizer Matrix: ' + str(check_shape.shape))
print('\n\n')
shape_df = pd.DataFrame(check_shape)
shape_df.head()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Shape of the Count Vectorizer Matrix: (95641, 10619)





,0,1,2,3,4,5,6,7,8,9,...,10609,10610,10611,10612,10613,10614,10615,10616,10617,10618
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
cv_elements = 450  ##TODO

cv = CountVectorizer(max_features = cv_elements)
X = cv.fit_transform(corpus).toarray()

In [12]:
# datasubset = dataset.head(1000)
datasubset = dataset
print(len(datasubset))

95641


In [13]:
# Creating the outout vector:

# y = datasubset.iloc[:, 1].values
y = datasubset.Sentiment

# Splitting the dataset into training and testing sets
# Setting 80% Training, 20% testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [14]:
from sklearn.ensemble import RandomForestClassifier


no_of_trees = 50 
no_of_features = 100 


classifier = RandomForestClassifier(n_estimators = no_of_trees, criterion = 'entropy', max_features = no_of_features, random_state = 0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
accuracy_rate =(cm[0][0]+cm[1][1])/(cm[0][1]+cm[1][0]+cm[0][0]+cm[1][1])
print('\n')

print("Accuray Rate with Rando Forest classifier= " + str(accuracy_rate))



Accuray Rate with Rando Forest classifier= 0.6862416107382551


In [15]:
print ("Accuracy random forest = "+ str(np.mean(y_pred == y_test)))
# my_tags = ['pos','neg']
# my_tags = ['pos','neg','neutral']
my_tags = ['neg','somehow neg','neutral','somehow pos','pos']
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred,
    target_names=my_tags))

Accuracy random forest = 0.5835642218620942
              precision    recall  f1-score   support

         neg       0.38      0.17      0.23       822
 somehow neg       0.45      0.21      0.29      3224
     neutral       0.63      0.88      0.73     10072
 somehow pos       0.48      0.32      0.38      3926
         pos       0.41      0.24      0.30      1085

   micro avg       0.58      0.58      0.58     19129
   macro avg       0.47      0.36      0.39     19129
weighted avg       0.55      0.58      0.54     19129

